In [ ]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"

RANDOM_SEED = 42 # for reproducibility

# Preparazione

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5efd7476fa946042f38fa85a6eb131d5fe154e38cb6bffe57668ff644f45ca04
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2Tn9uykXlYRjLcSLIgmv1iLdEvG_3gqA9M3V4hLGqH9N3JWEr

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!apt install openjdk-8-jdk-headless -qq

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 120828 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u382-ga-1~22.04.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u382-ga-1~22.04.1) ...

In [ ]:
from pyngrok import ngrok
import os
import re

In [ ]:
os.environ["JAVA_HOME"] = JAVA_HOME

In [ ]:
# Connect this colab to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Install PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=5656de4688c15e45a678bc0f6b4e941882e8981d25cee5761aa26e0e50b6529b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# Import libraries
# POI CAPISCI QUALE TI SERVE VERAMENTE
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as sqlf
from pyspark import SparkContext, SparkConf
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.ml.feature import HashingTF, CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Normalizer
from pyspark.ml.evaluation import ClusteringEvaluator
import gc

In [ ]:
# Create the session
# POI CAPISCI SE SONO I PARAMETRI GIUSTI


# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set('setCheckpointDir', '/content/gdrive/MyDrive/VideogameRecommenderSystem/checkpoints_pyspark').\
                setAppName("VideogameRecommenderSystem").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
port = '4050'
public_url = ngrok.connect(port).public_url

In [ ]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://4357-35-233-202-70.ngrok-free.app" -> "http://127.0.0.1:4050"


In [ ]:
spark

In [ ]:
games_df = spark.read.load('/content/gdrive/MyDrive/VideogameRecommenderSystem/games.csv',
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true")

In [ ]:
print("Row number:", games_df.count(), "\nColumn number:", len(games_df.columns))

Row number: 131974 
Column number: 6


In [ ]:
#games_df.show(n=5, truncate=False)

In [ ]:
#games_df.summary('count').show()

In [ ]:
#print("Row number before excluding rows with NaNs:", games_df.count())

In [ ]:
games_df = games_df.dropna()
#print("Row number after excluding rows with NaNs:", games_df.count())

In [ ]:
# Remove the links from the games descriptions since they are not relevant for
# the project
CLEANER_LINKS = re.compile('http[s]?://\S+')

def clean_links(raw_text):
  cleantext = re.sub(CLEANER_LINKS, '', raw_text)
  return cleantext

In [ ]:
# SPIEGA PERCHÉ HAI BISOGNO DI STO UDF
clean_links_UDF = sqlf.udf(lambda x : clean_links(x), StringType())

In [ ]:
games_df = games_df.withColumn('about_the_game', clean_links_UDF(sqlf.col('about_the_game')))
games_df = games_df.withColumn('about_the_game', sqlf.trim(sqlf.col('about_the_game')))
games_df = games_df.withColumn('short_description', clean_links_UDF(sqlf.col('short_description')))
games_df = games_df.withColumn('short_description', sqlf.trim(sqlf.col('short_description')))

In [ ]:
#games_df.show(n=5, truncate=False)

In [ ]:
# ELIMINA GIOCHI CON 0 RECENSIONI

games_df = games_df.where(games_df.total_recommendations > 0)

In [ ]:
games_df.count()

15861

In [ ]:
#cluster_games_df = games_df.drop('total_recommendations', 'about_the_game')
cluster_games_df = games_df.drop('total_recommendations', 'short_description')

In [ ]:
#cluster_games_df = cluster_games_df.withColumn('short_description', sqlf.lower(sqlf.col('short_description')))
cluster_games_df = cluster_games_df.withColumn('about_the_game', sqlf.lower(sqlf.col('about_the_game')))

In [ ]:
CLEANER_PUNCTUATION = re.compile('[^\w\s]')

def clean_punctuation(raw_text):
  cleantext = re.sub(CLEANER_PUNCTUATION, ' ', raw_text)
  return cleantext

In [ ]:
clean_punctuation_UDF = sqlf.udf(lambda x : clean_punctuation(x), StringType())

In [ ]:
#cluster_games_df = cluster_games_df.withColumn('short_description', clean_punctuation_UDF(sqlf.col('short_description')))
cluster_games_df = cluster_games_df.withColumn('about_the_game', clean_punctuation_UDF(sqlf.col('about_the_game')))

In [ ]:
CLEANER_EXTRA_SPACES = re.compile('\s\s+')

def clean_extra_spaces(raw_text):
  cleantext = re.sub(CLEANER_EXTRA_SPACES, ' ', raw_text)
  return cleantext

In [ ]:
clean_extra_spaces_UDF = sqlf.udf(lambda x : clean_extra_spaces(x), StringType())

In [ ]:
#cluster_games_df = cluster_games_df.withColumn('short_description', clean_extra_spaces_UDF(sqlf.col('short_description')))
cluster_games_df = cluster_games_df.withColumn('about_the_game', clean_extra_spaces_UDF(sqlf.col('about_the_game')))

In [ ]:
#tokenizer_short_description = Tokenizer(inputCol='short_description', outputCol="tokens_short_description")
#tokens_short_description_df = tokenizer_short_description.transform(cluster_games_df)
#tokens_short_description_df = tokens_short_description_df.drop('about_the_game')

tokenizer_about_the_game = Tokenizer(inputCol='about_the_game', outputCol="tokens_about_the_game")
tokens_about_the_game_df = tokenizer_about_the_game.transform(cluster_games_df)
tokens_about_the_game_df = tokens_about_the_game_df.drop('short_description')

In [ ]:
#stopwords_remover_short_description = StopWordsRemover(inputCol="tokens_short_description", outputCol="terms_short_description")
#terms_short_description_df = stopwords_remover_short_description.transform(tokens_short_description_df)

stopwords_remover_about_the_game = StopWordsRemover(inputCol="tokens_about_the_game", outputCol="terms_about_the_game")
terms_about_the_game_df = stopwords_remover_about_the_game.transform(tokens_about_the_game_df)

In [ ]:
stemmer = SnowballStemmer(language="english")
stemmer_udf = sqlf.udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))

In [ ]:
#terms_short_description_stemmed_df = terms_short_description_df.withColumn("terms_short_description_stemmed", stemmer_udf("terms_short_description"))

terms_short_description_stemmed_df = terms_about_the_game_df.withColumn("terms_about_the_game_stemmed", stemmer_udf("terms_about_the_game"))

In [ ]:
#cv = CountVectorizer(inputCol='terms_short_description_stemmed', outputCol="tf_features", vocabSize=1000, minDF=10)

cv = CountVectorizer(inputCol='terms_about_the_game_stemmed', outputCol="tf_features", vocabSize=1000, minDF=10)
idf = IDF(inputCol="tf_features", outputCol="features")

pipeline = Pipeline(stages=[cv, idf])
features = pipeline.fit(terms_short_description_stemmed_df)
tf_idf_df = features.transform(terms_short_description_stemmed_df)

In [ ]:
@sqlf.udf("long")
def num_nonzeros(v):
  return v.numNonzeros()

In [ ]:
tf_idf_df = tf_idf_df.where(num_nonzeros("features") > 0)

In [ ]:
#tf_idf_df.count()

# Funzioni

In [ ]:
def k_means(dataset,
            n_clusters,
            features_col='features',
            prediction_col='cluster',
            random_seed=RANDOM_SEED):

  print("""K-means parameter:
  - K (number of clusters) = {:d}
  - random seed = {:d}
  """.format(n_clusters, random_seed))

  dataset = Normalizer(inputCol=features_col, outputCol=features_col+"_norm", p=1).transform(dataset)
  features_col = features_col+"_norm"

  k_means = KMeans(
      featuresCol=features_col,
      predictionCol=prediction_col,
      k=n_clusters,
      initMode='k-means||',
      initSteps=5,
      tol=0.000001,
      maxIter=20,
      seed=random_seed,
      distanceMeasure='cosine'
      )
  model = k_means.fit(dataset)

  clusters_df = model.transform(dataset)

  return model, clusters_df

In [ ]:
def evaluate_k_means(clusters,
                     metric_name="silhouette",
                     distance_measure="cosine",
                     prediction_col="cluster"
                     ):

  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure,
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)

# Clustering

In [ ]:
k = 100
print("Running K-means using K = {:d}".format(k))
model, clusters_df = k_means(tf_idf_df, k)
silhouette_k = evaluate_k_means(clusters_df)
wssd_k = model.summary.trainingCost
print("Silhouette coefficient computed with cosine distance: {:.3f}".format(silhouette_k))
print("Within-cluster Sum of Squared Distances (using cosine distance): {:.3f}".format(wssd_k))
print("--------------------------------------------------------------------------------------")

In [ ]:
path = '/content/gdrive/MyDrive/VideogameRecommenderSystem/models/KMEANS_about_K_{}'.format(k)
model.save(path)